# Lesson - SQL Multi-column Indexing

In this lesson, we'll explore how to create indexes for speeding up queries that filter on multiple columns.

We'll work with `factbook.db`, a SQLite database that contains information about each country in the world. This database contains just the `facts` table and each row represents a single country. This version of `factbook.db` contains pop_idx already created.

Here are some of the columns:

`name` -- the name of the country.
`area` -- the total land and sea area of the country.
`population` -- the population of the country.
`birth_rate` -- the birth rate of the country.
`created_at` -- the date the record was created.
`updated_at` -- the date the record was updated.

Previously we limited ourselves to working with queries that only filtered on one column like:
`SELECT * FROM facts WHERE name = 'India';`

In this lesson, we'll explore how to create indexes for speeding up queries that filter on multiple columns, like:
`SELECT * FROM facts WHERE population > 1000000 AND population_growth < 2.0;`
We'll also explore how to modify the queries we write to better take advantage of indexes. For example, if we create an index for the `name` column, we'll explore why the following query:

`SELECT name from facts WHERE name = 'India';`
will be faster than:
`SELECT * from facts WHERE name = 'India'`;`


### QUERY PLAN
**Exercise**
- Return the query plan for a query that returns all rows where population is greater than 1000000 and where population_growth is less than 0.05.
- We're interested in all of the columns in the rows.
- Assign the query plan to query_plan_one and use print function to display the query plan.

In [3]:
import sqlite3
conn = sqlite3.connect("factbook.db")

query_one="EXPLAIN QUERY PLAN SELECT * FROm facts WHERE (population>1000000 AND population_growth<0.05);"
query_plan_one=conn.execute(query_one).fetchall()

print(query_plan_one)

[(3, 0, 0, 'SEARCH TABLE facts USING INDEX pop_idx (population>?)')]


### Query Plan for Multi-column Queries

We will now add index for population_growth column to see how SQLite uses these indexes for returning the same query. For exercise purposes we will also again try to add pop_idx which already exists conditioned bny `IF NOT ExISTS` statemet.

**Exercise**

- Create an index called `pop_idx` for the `population` column in the facts table.
- Create an index called `pop_growth_idx` for the `population_growth` column in the `facts` table.
- Return the query plan for a query that returns all rows where population is greater than 1000000 and where `population_growth` is less than 0.05. We're interested in all of the columns in the rows.
- Assign the query plan to `query_plan_two` and display it using the `prin`t function.

In [7]:
import sqlite3
conn = sqlite3.connect("factbook.db")

conn.execute("CREATE INDEX IF NOT EXISTS pop_idx ON facts(population);")

conn.execute("CREATE INDEX IF NOT EXISTS pop_growth_idx ON facts(population_growth);")
            
query_two="EXPLAIN QUERY PLAN SELECT * FROM facts WHERE (population>1000000 AND population_growth<0.05);"
query_plan_two=conn.execute(query_two).fetchall()

print(query_plan_two)

[(3, 0, 0, 'SEARCH TABLE facts USING INDEX pop_growth_idx (population_growth<?)')]


**Explanation of Results**

`SQLite` returns only a high-level query plan when we use the `EXPLAIN QUERY PLAN` statement in front of a query. This means that we'll often have to augment the returned query plan with your own understanding of the available indexes. In this case, the facts table has 2 indexes:

- one ordered by `population` called `pop_idx`,
- one ordered by `population_growth`, called `pop_growth_idx`.

SQLite struggles to take advantage of both indexes since each index is optimized for lookups on just that column. SQLite can use the indexes to quickly find the row `id` values where **either** population is greater than 1000000 or where population_growth is less than 0.05. If SQLite uses the index of population values to return all of the row `id` values where population is less than 1000000, it can't use those id values to search the `pop_growth_idx` index quickly to find the rows where `population_growth` is less than 0.05.

If we look at the query plan, we can infer that `SQLite` first decided to use the `pop_growth_idx` index to return the `id` values for the rows where population_growth was less than 0.05. Then, SQLite used a `binary search` on the `facts` table to access the row at each `id` value, add that row to a temporary collection if the value for population was greater than 1000000, and return the collection of rows.

The reason why `SQLite` chose the `pop_growth_idx` instead of the `pop_idx`is, because when there are 2 possible indexes available, `SQLite` tries to estimate which index will result in better performance. To keep SQLite lightweight, limited ability was added to estimate and plan accurately and `SQLite` often ends up picking an index at random.

### Multi-column Index

In cases like above, we need to create a multi-column index that contains values from both of the columns we're filtering on. This way, both criteria in the `WHERE` statement can be evaluted in the index itself and the `facts` table will only be queried at the end when we have the specific row `id` values.

While the single column indexes contain just the primary key column (`population`) and the row `id` (id) columns, the multi-column index contains the `population_growth` column as well. SQLite can:

- use binary search to find the first row in this index where population is greater than 1000000,
- add the row to a `temporary collection` if `population_growth` is less than 0.05,
- advance to the next row (the index is ordered by population),
- add the row to a temporary collection if population_growth is less than 0.05,
- when the end of the index is reached, look up each row in `facts` using the `id` values from the `temporary collection`.

This way the `facts` table is only accessed at the end and the index is used to process the `WHERE` criteria.
When creating a multi-column index, we need to specify which of the columns we want as the `primary key`. In the example above, this means that `SQLit`e can use `binary search` to quickly jump to the first row that matches a specific population value but not before identifyingg it with the first row that matches a specific population_growth value.

### Creating a Multi-column Index

To create a multi-column index, we use the same `CREATE INDEX` syntax as before but instead specify 2 columns in the `ON` statement:

```
CREATE INDEX index_name ON table_name(column_name_1, column_name_2);
```
**First column in the parentheses becomes the primary key for the index.**

**Exercise**
- Create a multi-column index for `population` and `population_growth` named `pop_pop_growth_idx` with `population` as the `primary key`.
- Return the query plan for a query that returns all rows where `population` is greater than 1000000 and where `population_growth` is less than 0.05. We're interested in all of the columns in the rows.
- Assign the returned query plan to `query_plan_three` and use the `print` function to display it.

In [1]:
import sqlite3
conn = sqlite3.connect("factbook.db")

conn.execute("CREATE INDEX IF NOT EXISTS pop__pop_growth_idx ON facts(population,population_growth);")

         
query_three="EXPLAIN QUERY PLAN SELECT * FROM facts WHERE (population>1000000 AND population_growth<0.05);"
query_plan_three=conn.execute(query_three).fetchall()

print(query_plan_three)

[(3, 0, 0, 'SEARCH TABLE facts USING INDEX pop__pop_growth_idx (population>?)')]


### Covering Index

Above `SQLite` used the multi-column index `pop_pop_growth_idx` that we created instead of either `pop_growth_idx` or `pop_idx`. `SQLite` only needed to access the `facts` table to return the rest of the column values for the rows that met the `WHERE` criteria. This is only because the `pop_pop_growth_idx` doesn't contain the other values (besides population and population_growth).

If we restricted the columns in the `SELECT` that we want returned to just `population` and `population_growth`, SQLite will not need to interact with the `facts` table since the `pop_pop_growth_idx` can service the query.

When an index contains all of the information necessary to answer a query, it's called a **covering index**. Since the index covers for the actual table and can return the requested results to the query, `SQLite` doesn't need to query the actual table. For many queries, especially as our data gets larger, this can be much more efficient.

**Exercise**

- Return the query plan for a query that returns all rows where `population` is greater than 1000000 and where `population_growth` is less than 0.05. Select only the `population` and `population_growth columns`.
- Assign the returned query plan to query_plan_four and use the print function to display it.

In [2]:
conn = sqlite3.connect("factbook.db")

query_four="EXPLAIN QUERY PLAN SELECT population, population_growth FROM facts WHERE (population>1000000 AND population_growth<0.05);"
query_plan_four=conn.execute(query_four).fetchall()

print(query_plan_four)

[(2, 0, 0, 'SEARCH TABLE facts USING COVERING INDEX pop__pop_growth_idx (population>?)')]


### Covering Index for Single Column

two things that stand out from the query plan from the previous screen:

- instead of `USING INDEX` the query plan says `USING COVERING INDEX`,
- the query plan still contains `SEARCH TABLE facts` as before.

Even though the query plan indicates that a `binary search` on facts was performed, this is misleading and it was instead able to use the covering index. We can read more about that on the [documentation](https://www.sqlite.org/queryplanner.html#covidx).

Covering indexes don't apply just to multi-column indexes. If a query we write only touches a column in the database that we have a single-column index for, `SQLite` will use only the index to service the query. Let's test this by writing a query that can take advantage of just the index, `pop_idx`, for the `population` column.

**Exercise**

- Return the query plan for a query that returns all rows where `population` is greater than 1000000. We're only interested in the `population` column.
- Assign the returned query plan to query_plan_five and use the print function to display it.

In [3]:
conn = sqlite3.connect("factbook.db")

query_five="EXPLAIN QUERY PLAN SELECT population FROM facts WHERE population>1000000;"
query_plan_five=conn.execute(query_five).fetchall()

print(query_plan_five)

[(2, 0, 0, 'SEARCH TABLE facts USING COVERING INDEX pop_idx (population>?)')]


Since only the `population` values were necessary to service the query, `SQLite` used the `pop_idx` index as a covering index and didn't have to access the `facts` table.